In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import pandas as pd
import re

#this creates a list of each link

parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
#make sure that the below link is the one with all the schools in a list
resp = urllib.request.urlopen("https://reports.aashe.org/institutions/participants-and-reports/")
soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

most_recent_reports_links = []
head_link = "https://reports.aashe.org"
for link in soup.find_all('a', href=True):
        if link['href'][-18:-11] == "report/":
            most_recent_reports_links.append(head_link + link['href'])



In [2]:
(most_recent_reports_links) #showing what this list of links looks like

['https://reports.aashe.org/institutions/arizona-state-university-az/report/2020-03-05/',
 'https://reports.aashe.org/institutions/colorado-state-university-co/report/2019-12-06/',
 'https://reports.aashe.org/institutions/cornell-university-ny/report/2021-03-04/',
 'https://reports.aashe.org/institutions/stanford-university-ca/report/2019-02-22/',
 'https://reports.aashe.org/institutions/state-university-of-new-york-college-of-environmental-science-and-forestry-ny/report/2021-03-05/',
 'https://reports.aashe.org/institutions/thompson-rivers-university-bc/report/2018-05-31/',
 'https://reports.aashe.org/institutions/university-of-california-berkeley-ca/report/2021-03-04/',
 'https://reports.aashe.org/institutions/university-of-california-irvine-ca/report/2021-08-11/',
 'https://reports.aashe.org/institutions/university-of-connecticut-ct/report/2020-03-06/',
 'https://reports.aashe.org/institutions/university-of-new-hampshire-nh/report/2021-08-16/',
 'https://reports.aashe.org/institutio

In [3]:
#this pulls the school name from the list of links

school_names = []
for item in most_recent_reports_links:
    school_name = item[:-19]
    final_name = school_name[39:]
    final_name = final_name.replace("-", " ")
    
    school_names.append(final_name)
    
school_names
(school_names)

['arizona state university az',
 'colorado state university co',
 'cornell university ny',
 'stanford university ca',
 'state university of new york college of environmental science and forestry ny',
 'thompson rivers university bc',
 'university of california berkeley ca',
 'university of california irvine ca',
 'university of connecticut ct',
 'university of new hampshire nh',
 'universite de sherbrooke qc',
 'agnes scott college ga',
 'american university dc',
 'appalachian state university nc',
 'babson college ma',
 'bard college ny',
 'bates college me',
 'belmont university tn',
 'bennington college vt',
 'berea college ky',
 'binghamton university ny',
 'bowdoin college me',
 'bucknell university pa',
 'california polytechnic state university ca',
 'california state university channel islands ca',
 'california state university chico ca',
 'california state university northridge ca',
 'california state university sacramento ca',
 'california state university san marcos ca',
 'ca

In [4]:
# this scrapes and pivots every item in the table
#then it appends to the end of the pandas data frame for each school

In [5]:
#the cell here takes 3 seconds for every link in the most recent reports
#that means for the current length(555) it takes about 28 minutes to run 
#So in the meantime make a cup of coffee, check your email, watch an episode of the office and stretch your legs

df_list = []
total_df = pd.DataFrame()

for idx in range(len((most_recent_reports_links))):
    # Site URL
    url=most_recent_reports_links[idx]

    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text

    # Parse HTML code for the entire site
    soup = BeautifulSoup(html_content, "lxml")
    #print(soup.prettify()) # print the parsed data of html

    credit = soup.find_all("table", attrs={"class": "table table-condensed table-striped"})

    table = []

    for i in range(0,len(credit)):
        table1 = credit[i]
        # the head will form our column names
        body = table1.find_all("tr")
        # Head values (Column names) are the first items of the body list
        head = body[0] # 0th item is the header row
        body_rows = body[1:] # All other items becomes the rest of the rows

        # Lets now iterate through the head HTML code and make list of clean headings

        # Declare empty list to keep Columns names
        headings = []
        for item in head.find_all("th"): # loop through all th elements
            # convert the th elements to text and strip "\n"
            item = (item.text).rstrip("\n")
            # append the clean column name to headings
            headings.append(item)

        all_rows = [] # will be a list for list for all rows
        for row_num in range(len(body_rows)): # A row at a time
            row = [] # this will hold entries for one row
            for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
                # row_item.text removes the tags from the entries
                # the following regex is to remove \xa0 and \n and comma from row_item.text
                # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
                aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
                #append aa to row - note one row entry is being appended
                row.append(aa)
            # append one row to all_rows

            if len(row)> 3:
                row = row[:3]

            all_rows.append(row)
        df = pd.DataFrame(data=all_rows,columns=headings)

            #df = df.set_index("Credit")
        df = df.drop(columns = ["Status"], axis = 1)
        df = df.T
        table.append(df)
        
#this pivots the table so that each category is a column
    final_df = pd.concat(table, axis = 1, ignore_index = True)
    final_df.columns = final_df.iloc[0]

    #the block below turns the string value into float for numerical use
    for item in list(final_df.columns):
        try:
            no_spaces=(final_df[item].iloc[1].replace(" ", ""))
            slash_loc = no_spaces.find("/")
            value = float(no_spaces[0:slash_loc])
            final_df[item].iloc[1] = value
        except:
            continue
            
            
    final_df = final_df.drop(index = "Credit")
    final_df = final_df.rename(index = {"Points": school_names[idx]})
    final_df["Link"] = url
    
    total_df = pd.concat([total_df, final_df])
    


In [6]:
total_df

Credit,Executive Letter,Points of Distinction,Institutional Boundary,Operational Characteristics,Academics and Demographics,Academic Courses,Learning Outcomes,Undergraduate Program,Graduate Program,Immersive Experience,...,Committee on Socially Responsible Investment,Student-Managed SRI Fund,Socially Responsible Investment Policy,Campus Water Balance,Food Purchasing,Post-Consumer Food Waste Composting,Carpool Matching,Climate Plan,Support Programs for Under-Represented Groups,Health and Safety Management Certification
arizona state university az,NaN,NaN,NaN,NaN,NaN,11.98,7.33,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
colorado state university co,NaN,NaN,NaN,NaN,NaN,14.0,8.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cornell university ny,NaN,NaN,NaN,NaN,NaN,12.44,7.37,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stanford university ca,NaN,NaN,NaN,NaN,NaN,12.2,8.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
state university of new york college of environmental science and forestry ny,NaN,NaN,NaN,NaN,NaN,14.0,8.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
illinois institute of technology il,NaN,NaN,NaN,NaN,NaN,3.18,0.0,3.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
simon fraser university bc,NaN,NaN,NaN,NaN,NaN,12.75,2.02,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
university of manitoba mb,NaN,NaN,NaN,NaN,NaN,12.79,4.91,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
university of washington seattle wa,NaN,NaN,NaN,NaN,NaN,12.15,2.14,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#this drops the empty columns and turns the NaNs into 0's
total_df = total_df.dropna(axis=1, how='all')
total_df= total_df.drop(columns= ['Physical Risk QUANTITY','Low and Low to Medium Risk', 'Medium to High Risk', 'High and Extremely High Risk'])
total_df = total_df.fillna(0)
total_df

Credit,Academic Courses,Learning Outcomes,Undergraduate Program,Graduate Program,Immersive Experience,Sustainability Literacy Assessment,Incentives for Developing Courses,Campus as a Living Laboratory,Research and Scholarship,Support for Sustainability Research,...,Community Stakeholder Engagement,Hospital Network,Center for Sustainability Across the Curriculum,Academy-Industry Connections,Green Cleaning Certification,Fleet Certification,Laboratory Animal Welfare,Energy System Certification,Campus Water Balance,Health and Safety Management Certification
arizona state university az,11.98,7.33,3.0,3.0,2.0,4.0,2.0,4.0,12.00,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
colorado state university co,14.00,8.00,3.0,3.0,2.0,4.0,2.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cornell university ny,12.44,7.37,3.0,3.0,2.0,4.0,2.0,4.0,12.00,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
stanford university ca,12.20,8.00,3.0,3.0,2.0,4.0,2.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
state university of new york college of environmental science and forestry ny,14.00,8.00,3.0,3.0,2.0,4.0,2.0,4.0,12.00,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
illinois institute of technology il,3.18,0.00,3.0,3.0,0.0,0.0,0.0,2.0,7.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
simon fraser university bc,12.75,2.02,3.0,3.0,2.0,0.0,2.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
university of manitoba mb,12.79,4.91,3.0,3.0,2.0,0.0,0.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
university of washington seattle wa,12.15,2.14,3.0,3.0,2.0,4.0,2.0,4.0,8.03,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
total_df["sum"] = total_df.sum(axis=1) #only run this once, Otherwise it will continue to sum ontop


In [9]:
mess_around  = total_df
total_df

Credit,Academic Courses,Learning Outcomes,Undergraduate Program,Graduate Program,Immersive Experience,Sustainability Literacy Assessment,Incentives for Developing Courses,Campus as a Living Laboratory,Research and Scholarship,Support for Sustainability Research,...,Hospital Network,Center for Sustainability Across the Curriculum,Academy-Industry Connections,Green Cleaning Certification,Fleet Certification,Laboratory Animal Welfare,Energy System Certification,Campus Water Balance,Health and Safety Management Certification,sum
arizona state university az,11.98,7.33,3.0,3.0,2.0,4.0,2.0,4.0,12.00,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.95
colorado state university co,14.00,8.00,3.0,3.0,2.0,4.0,2.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,174.80
cornell university ny,12.44,7.37,3.0,3.0,2.0,4.0,2.0,4.0,12.00,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,169.48
stanford university ca,12.20,8.00,3.0,3.0,2.0,4.0,2.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,175.03
state university of new york college of environmental science and forestry ny,14.00,8.00,3.0,3.0,2.0,4.0,2.0,4.0,12.00,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
illinois institute of technology il,3.18,0.00,3.0,3.0,0.0,0.0,0.0,2.0,7.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.32
simon fraser university bc,12.75,2.02,3.0,3.0,2.0,0.0,2.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.79
university of manitoba mb,12.79,4.91,3.0,3.0,2.0,0.0,0.0,4.0,12.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136.54
university of washington seattle wa,12.15,2.14,3.0,3.0,2.0,4.0,2.0,4.0,8.03,0.0,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,151.83


In [11]:
mess_around = mess_around.sort_values(['sum'], ascending = False)
#sorts by the highest scoring school first

In [12]:
#make sure the directory and file name are in the format you want them to be in
mess_around.to_csv("/Users/jacobdudley/Desktop/final_file.csv")
